# CVD Divergence Strategy Research
This notebook is for analyzing and testing the CVD Divergence indicator before backtesting.

In [ ]:
# Import QuantBook for research
from AlgorithmImports import *
import pandas as pd
import matplotlib.pyplot as plt

qb = QuantBook()

In [ ]:
# Add MES futures data
future = qb.add_future(Futures.Indices.SP500EMini, Resolution.MINUTE)
future.set_filter(0, 90)

In [ ]:
# Fetch historical data
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 31)

history = qb.history(future.symbol, start_date, end_date, Resolution.MINUTE)
print(f"Fetched {len(history)} bars")
history.head()

In [ ]:
# Calculate CVD manually for visualization
def calculate_cvd(df, period=21):
    """Calculate Cumulative Volume Delta."""
    price_range = df['high'] - df['low']
    
    # Avoid division by zero
    price_range = price_range.replace(0, 0.0001)
    
    buying_volume = df['volume'] * ((df['close'] - df['low']) / price_range)
    selling_volume = df['volume'] * ((df['high'] - df['close']) / price_range)
    
    delta = buying_volume - selling_volume
    cvd = delta.rolling(window=period).sum()
    
    return cvd

# Add CVD to dataframe
if len(history) > 0:
    history['cvd'] = calculate_cvd(history, period=21)
    history['ema_50'] = history['close'].ewm(span=50).mean()
    print("CVD calculated successfully")

In [ ]:
# Visualize price and CVD
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot price
ax1.plot(history.index, history['close'], label='Close Price', color='blue')
ax1.plot(history.index, history['ema_50'], label='EMA 50', color='orange', alpha=0.7)
ax1.set_ylabel('Price')
ax1.set_title('MES Futures Price')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot CVD
cvd_positive = history['cvd'].where(history['cvd'] > 0, 0)
cvd_negative = history['cvd'].where(history['cvd'] < 0, 0)

ax2.bar(history.index, cvd_positive, color='green', alpha=0.5, label='Positive CVD')
ax2.bar(history.index, cvd_negative, color='red', alpha=0.5, label='Negative CVD')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('CVD')
ax2.set_title('Cumulative Volume Delta (21-period)')
ax2.set_xlabel('Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Test ATR indicator
atr = qb.indicator(AverageTrueRange(55), future.symbol, 100, Resolution.MINUTE)
print(f"ATR Value: {atr.current.value}")

In [ ]:
# Display summary statistics
print("\n=== Summary Statistics ===")
print(f"Price Range: ${history['close'].min():.2f} - ${history['close'].max():.2f}")
print(f"CVD Range: {history['cvd'].min():.2f} - {history['cvd'].max():.2f}")
print(f"Average Volume: {history['volume'].mean():.0f}")
print(f"Total Bars: {len(history)}")